In [1]:
from datasets import load_dataset
squad = load_dataset("squad")
shuffled_squad = squad.shuffle()

Reusing dataset squad (C:\Users\GZK\.cache\huggingface\datasets\squad\plain_text\1.0.0\d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)


  0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
train_contexts = []
train_questions = []
train_answers = []
train_ids = []


for i in range(len(shuffled_squad['train'])):
    train_contexts.append(shuffled_squad['train'][i]['context'])
    train_questions.append(shuffled_squad['train'][i]['question'])
    train_answers.append(shuffled_squad['train'][i]['answers'])
    train_ids.append(shuffled_squad['train'][i]['id'])




In [3]:
validation_contexts = train_contexts[77599:87599]
train_contexts = train_contexts[:77599]
validation_question = train_questions[77599:87599]
train_questions = train_questions[:77599]
validation_answers = train_answers[77599:87599]
train_answers = train_answers[:77599]
validation_ids = train_ids[77599:87599]
train_ids = train_ids[:77599]




In [6]:
train_contexts = train_contexts[:10]
train_questions = train_questions[:10]
train_answers = train_answers[:10]
train_ids = train_ids[:10]


In [7]:
import nltk as tk
import re
sep_train_contexts = []
sep_train_questions = []
sep_train_answers = []
error_index = []
null_answer = {'text': '[NULL]', 'answer_start': 0}
for i in range(len(train_contexts)):
    tokens = tk.sent_tokenize(train_contexts[i])
    for token in tokens:
        if train_answers[i]['text'][0] in token:
            try:
                answer_start = re.search(train_answers[i]['text'][0], token)
                answer = {'text': train_answers[i]['text'], 'answer_start':  answer_start.span()[0]}
                sep_train_contexts.append(token)

            
                sep_train_answers.append(answer)
                sep_train_questions.append(train_questions[i])
            except:
                error_index.append(i)

                # print(i)
        # else:
        #     sep_train_contexts.append('[NULL]' + token)
        #     sep_train_answers.append(null_answer)
        #     sep_train_questions.append(train_questions[i])
print("error_index : ", len(error_index))
            


error_index :  0


In [8]:
def add_end_idx(answers, contexts):
    for answer, context in zip(answers, contexts):
        gold_text = answer['text'][0]
        if type(answer['answer_start']) == list:
            temp = answer['answer_start'][0]
            answer['answer_start'] = temp
        else:
            start_idx = answer['answer_start']
        end_idx = start_idx + len(gold_text)

        # sometimes squad answers are off by a character or two – fix this
        if context[start_idx:end_idx] == gold_text:
            answer['answer_end'] = end_idx
        elif context[start_idx-1:end_idx-1] == gold_text:
            answer['answer_start'] = start_idx - 1
            answer['answer_end'] = end_idx - 1     # When the gold label is off by one character
        elif context[start_idx-2:end_idx-2] == gold_text:
            answer['answer_start'] = start_idx - 2
            answer['answer_end'] = end_idx - 2     # When the gold label is off by two characters
        else:
            answer['answer_end'] = end_idx

add_end_idx(sep_train_answers, sep_train_contexts)



In [10]:
add_end_idx(train_answers, train_contexts)

In [11]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

sep_train_encodings = tokenizer(sep_train_contexts, sep_train_questions, truncation=True, padding=True)


In [12]:
train_encodings = tokenizer(train_contexts, train_questions, truncation=True, padding=True)

In [13]:
def add_token_positions(encodings, answers):
    start_positions = []
    end_positions = []
    print("len len(answers) : ",len(answers))
    for i in range(len(answers)):
        start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))

        
        end_positions.append(encodings.char_to_token(i, answers[i]['answer_end'] - 1))
        # if None, the answer passage has been truncated
        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length
        if end_positions[-1] is None:
            end_positions[-1] = tokenizer.model_max_length
    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

add_token_positions(sep_train_encodings, sep_train_answers)


len len(answers) :  10


In [14]:
add_token_positions(train_encodings, train_answers)

len len(answers) :  10


In [15]:
import torch

class SquadDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

sep_train_dataset = SquadDataset(sep_train_encodings)


In [16]:
train_dataset = SquadDataset(train_encodings)

In [17]:
from transformers import DistilBertForQuestionAnswering
model = DistilBertForQuestionAnswering.from_pretrained("distilbert-base-uncased")

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this mode

In [28]:
import datetime
import json
import argparse
import collections

import numpy as np
import os
import re
import string
import sys
ref_token_id = tokenizer.pad_token_id # A token used for generating token reference
sep_token_id = tokenizer.sep_token_id # A token used as a separator between question and text and it is also added to the end of the text.
cls_token_id = tokenizer.cls_token_id
print (datetime.datetime.now())
def predict(inputs):
    output = model(inputs)
    return output.start_logits, output.end_logits


def construct_input_ref_pair(question, text, ref_token_id, sep_token_id, cls_token_id):
    question_ids = tokenizer.encode(question, add_special_tokens=False)
    text_ids = tokenizer.encode(text, add_special_tokens=False)

    # construct input token ids
    input_ids = [cls_token_id] + question_ids + [sep_token_id] + text_ids + [sep_token_id]

    # construct reference token ids
    ref_input_ids = [cls_token_id] + [ref_token_id] * len(question_ids) + [sep_token_id] + \
                    [ref_token_id] * len(text_ids) + [sep_token_id]

    return torch.tensor([input_ids], device=device), torch.tensor([ref_input_ids], device=device), len(question_ids)

def predict_qt(question, text):
    input_ids, ref_input_ids, sep_id = construct_input_ref_pair(question, text, ref_token_id, sep_token_id, cls_token_id)

    indices = input_ids[0].detach().tolist()
    all_tokens = tokenizer.convert_ids_to_tokens(indices)

    ground_truth = '13'


    start_scores, end_scores = predict(input_ids)


    return (' '.join(all_tokens[torch.argmax(start_scores) : torch.argmax(end_scores)+1]))
def make_qid_to_has_ans(dataset):
    qid_to_has_ans = {}
    for article in dataset:
        for p in article['paragraphs']:
            for qa in p['qas']:
                qid_to_has_ans[qa['id']] = bool(qa['answers'])
    return qid_to_has_ans

def get_raw_scores(dataset, preds):
    exact_scores = {}
    f1_scores = {}
    for article in dataset:
        for p in article['paragraphs']:
            for qa in p['qas']:
                qid = qa['id']
                gold_answers = [a['text'] for a in qa['answers']
                            if normalize_answer(a['text'])]
            if not gold_answers:
            # For unanswerable questions, only correct answer is empty string
                gold_answers = ['']
            if qid not in preds:
            # print('Missing prediction for %s' % qid)
                continue
            a_pred = preds[qid]
            # Take max over all gold answers
            exact_scores[qid] = max(compute_exact(a, a_pred) for a in gold_answers)
            f1_scores[qid] = max(compute_f1(a, a_pred) for a in gold_answers)
    return exact_scores, f1_scores
def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    def remove_articles(text):
        regex = re.compile(r'\b(a|an|the)\b', re.UNICODE)
        return re.sub(regex, ' ', text)
    def white_space_fix(text):
        return ' '.join(text.split())
    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)
    def lower(text):
        return text.lower()
    return white_space_fix(remove_articles(remove_punc(lower(s))))
def apply_no_ans_threshold(scores, na_probs, qid_to_has_ans, na_prob_thresh):
    new_scores = {}
    for qid, s in scores.items():
        pred_na = na_probs[qid] > na_prob_thresh
    if pred_na:
        new_scores[qid] = float(not qid_to_has_ans[qid])
    else:
        new_scores[qid] = s
    return new_scores
def test_valisdation():
    evl_dict = {}

    for i in range(len(validation_contexts)):
        question = validation_question[i]
        text = validation_contexts[i]
        if len(text) <= 512:
            answer = predict_qt(question, text)
            evl_dict[str(validation_ids[i])] = answer
    time = str(datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S'))
    json_dict = json.dumps(evl_dict)
    filname = time + "answers.txt"

    fo = open("./results/"+filname, "w",encoding='utf-8')
    fo.write(json_dict)
    fo.close()
    
    with open(r"./dev-v2.0.json") as f:
        dataset_json = json.load(f)
        dataset = dataset_json['data']
    preds = evl_dict
    
    na_probs = {k: 0.0 for k in preds}
    qid_to_has_ans = make_qid_to_has_ans(dataset)  # maps qid to True/False
    has_ans_qids = [k for k, v in qid_to_has_ans.items() if v]
    no_ans_qids = [k for k, v in qid_to_has_ans.items() if not v]
    exact_raw, f1_raw = get_raw_scores(dataset, preds)
    exact_thresh = apply_no_ans_threshold(exact_raw, na_probs, qid_to_has_ans,1.0)
    f1_thresh = apply_no_ans_threshold(f1_raw, na_probs, qid_to_has_ans,1.0)
    out_eval = make_eval_dict(exact_thresh, f1_thresh)
    if has_ans_qids:
        has_ans_eval = make_eval_dict(exact_thresh, f1_thresh, qid_list=has_ans_qids)
        merge_eval(out_eval, has_ans_eval, 'HasAns')
    if no_ans_qids:
        no_ans_eval = make_eval_dict(exact_thresh, f1_thresh, qid_list=no_ans_qids)
        merge_eval(out_eval, no_ans_eval, 'NoAns')

        print(json.dumps(out_eval, indent=2))

2022-03-17 17:16:16.574565


In [29]:
from torch.utils.data import DataLoader
from transformers import AdamW
import datetime

print (datetime.datetime.now())

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model.to(device)
model.train()

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

optim = AdamW(model.parameters(), lr=5e-5)
list_loss = []
for epoch in range(10):
    for batch in train_loader:
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_positions = batch['start_positions'].to(device)
        end_positions = batch['end_positions'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
        loss = outputs[0]
        loss.backward()
        optim.step()
    test_valisdation()
    list_loss.append(float(loss))
    model_name = "epoch" + str(epoch) + str(datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S')) + '.pth'
    torch.save(model, './models/' + model_name)

model.eval()

print (datetime.datetime.now())
print("loss : ", list_loss)

2022-03-17 17:16:17.281957


UnboundLocalError: local variable 'pred_na' referenced before assignment

In [17]:
from torch.utils.data import DataLoader
from transformers import AdamW
import datetime

print (datetime.datetime.now())

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model.to(device)
model.train()

train_loader = DataLoader(sep_train_dataset, batch_size=16, shuffle=True)

optim = AdamW(model.parameters(), lr=5e-5)
list_loss = []
for epoch in range(10):
    for batch in train_loader:
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_positions = batch['start_positions'].to(device)
        end_positions = batch['end_positions'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
        loss = outputs[0]
        loss.backward()
        optim.step()
    test_valisdation()
    list_loss.append(float(loss))
    model_name = "epoch" + str(epoch) + str(datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S')) + '.pth'
    torch.save(model, './models/' + model_name)

model.eval()

print (datetime.datetime.now())
print("loss : ", list_loss)

2022-03-17 05:04:41.246842
2022-03-17 10:14:14.660094
loss :  [0.4444069266319275, 0.6065800189971924, 0.16105425357818604, 0.08027053624391556, 0.20499780774116516, 0.3776370882987976, 0.09724403917789459, 0.5246251225471497, 0.08134955912828445, 0.0073407674208283424]
